# Overview 
In this notebook, we will show use the model as a judge pattern for evaluating chat / agent responses. We will be evaluating an agents ability to help users with finding and purchasing tickets to events.

In this notebook we will:
1. Define qualitative metrics that make sense for our usecase
1. Create a quality assurance rubric that an large language model (LLM) can use to grade qualitatively
1. Run our test Suite
1. Validate the results and understand what needs to change in the system to improve the overall user experience.


## Building the Evaluation Framework
The process for evaluating a chat "agent" is slightly different than single turn Q&A. On top of vending factually accurate results, we also care about the customer experience, tone, and whether the agent confirms with the user before purchasing tickets. In the example dataset, there are 50 made up chat conversations a human had with an chat model. To  use this for a different usecase, you'd want to gather 50+ examples from your agent and swap out the fake chat conversations with your own.

Resist the temptation to grab a premade list! Using your own questions from your use case will make a huge difference.

When evaluating a prompt, we want **at least** 50+ examples to run benchmarks on. One of the best ways to differentiate between a gen AI science project and a viable product is to count the number of automated tests. A handful of manual tests? Science Project. An automated system of hundreds of tests that runs every time you propose a change? Production ready.

**Note** In subsequent notebooks, we will show you how to compare conversations between two different chat models. For this notebook, we are just evaluating a single agent.


## Pre-Requisites

Pre-requisites
This notebook requires permissions to:

access Amazon Bedrock
access to Amazon OpenSearch Serverless

If running on SageMaker Studio, you should add the following managed policies to your role:

1. AmazonBedrockFullAccess
```

# Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

In [ ]:
# Install all the required dependencies if you haven't already done so from the requirements.txt

# %pip install -U boto3==1.34.82
# %pip install -U langchain==0.1.13
# %pip install -U pandas==2.2.2

In [ ]:
# Restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Create Eval Dataset

This part is a little tricky and time consuming. **For the purpose of this notebook, we went ahead and created a dataset**. We did this by prompting an LLM to generate fictitious chat conversations both good and bad.

We recommend currating additional examples by interacting with your chat bot to ensure a robust dataset. We also recommend you add to this eval dataset over time.


Next lets import our dataset into pandas and take a look at it!

In [ ]:
import json

lines = []
with open('../data/chat_evaluation_data.jsonl') as f:
    lines = f.read().splitlines()

chat_conversations = [json.loads(line) for line in lines]

print(chat_conversations[0])

# Define Metrics for Benchmarking

In this step, we'll be defining the metrics we care about in order to benchmark both different prompts and models. 

There are two main types of evaluation metrics. Qualitative and quantitative. Find descriptions of each type below. 

### Quantitative 
Quantitative metrics involve numerical measurements that can objectively compare different models. These typically include accuracy, perplexity, speed, and resource efficiency, among others. They provide a clear, standardized way to measure certain aspects of an LLM's performance, such as how well it predicts the next word in a sequence or how quickly it generates responses. 

### Qualitative
On the other hand, qualitative metrics assess the more subjective aspects of LLM performance, including the coherence, relevancy, creativity of generated text, and adherence to ethical guidelines. These are often evaluated through human judgment via methods such as expert reviews or user studies, offering insights into the user experience and the contextual appropriateness of the model's outputs. While quantitative metrics can offer precise, measurable benchmarks, qualitative metrics are crucial for understanding the nuances and real-world effectiveness of LLMs. 

For qualitative evals we will want to consider
1. Coherence of response
2. Relevance of information returned
3. Accuracy of response
4. Adherance to brand guidelines


## How do we gather qualitative metrics?

To gather qualitative metrics, we have two options. (1) Create a QA rubrik and give it to human evaluators or (2) Use that same rubrik and give it to an LLM to evaluate the responses. 

As a test suite gets larger, human evaluation becomes a bottleneck. Grading 500+ answers every time you make a change to a prompt is not scalable. Because of this, we'll opt to use an LLM to evaluate our responses. For poorly scoring responses, we can then manually check to see what's going on and fix the responses

## Lets create a grading prompt
Below you'll find a prompt that takes in the question, model response, correct answer, and a rubric that you will create to evaluate models output.

In [ ]:
# Fill in your rubric here.
RUBRIC = '''
- The conversation should have a friendly tone and not be overly verbose. 
- If the user asks to purchase a ticket, the assistant should get confirmation before executing a transaction.
- If a customer asks about something not related to an event, the model should respond indicating to the user that it cannot help them.
- The conversation should be less than 5 turns at most to complete the conversation. The  assistant should not get stuck asking the user to repeat things already discussed in the conversation.
- Any recommendation for events or artists should be relevant to the conversation.
'''

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages.base import BaseMessage

# We start by defining a "grader prompt" template.
def build_grader_prompt(conversation: str) -> BaseMessage:
    prompt = """You will be provided a chat conversation that an assistant had with a user, and a rubric that instructs you on what makes the conversation correct or incorrect.
    
    Here is the conversation.
    <conversation>
    {conversation}
    </conversation>
    
    Here is the rubric on how to grade the assistant response.
    <rubric>
    {rubric}
    </rubric>
    
    An answer is correct if it entirely meets the rubric criteria, and is otherwise incorrect.
    First, think through whether the answer is correct or incorrect based on the rubric inside <thinking></thinking> tags. Then, output either 'correct' if the answer is correct or 'incorrect' if the answer is incorrect inside <correctness></correctness> tags.
    """

    # First we will generate a prompt template using Langchain and the prompt above
    chat_template: ChatPromptTemplate = ChatPromptTemplate.from_messages([
        ("human", prompt)
    ])
        
    # Next, we will insert all the variables into into the prompt. 
    return chat_template.format_messages(
        conversation=conversation,
        rubric=RUBRIC
    ) 


# Helper Functions For Bedrock
In the section below we'll define some helper functions to speed up the evaluation process. We'll call bedrock from a threadpool

In [ ]:
SONNET_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
HAIKU_ID = "anthropic.claude-3-haiku-20240307-v1:0"

# To flip between different models, you can change these global variable.
MODEL_TO_USE = SONNET_ID

REGION = 'us-west-2'

# Helper Functions

from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import boto3
import time

from langchain_community.chat_models import BedrockChat
from langchain_core.messages.ai import AIMessage


def call_bedrock(request: BaseMessage):
    client = BedrockChat(
        model_id=MODEL_TO_USE, 
        model_kwargs= {"temperature": 0.5, "top_k": 500}
    )
    
    response = client.invoke(request)
    return response

# This is a bit funky. We're dumping all the requests into a thread pool
# And storing the index for the order in which they were submitted. 
# Lastly, we're inserting them into the response array at their index to ensure order.
def call_threaded(requests, function):
    # Dictionary to map futures to their position
    future_to_position = {}
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Submit all requests and remember their order
        for i, request in enumerate(requests):
            future = executor.submit(function, request)
            future_to_position[future] = i
        
        # Initialize an empty list to hold the responses
        responses = [None] * len(requests)
        
        # As each future completes, assign its result to the correct position
        for future in as_completed(future_to_position):
            position = future_to_position[future]
            try:
                response: AIMessage = future.result()
                responses[position] = response.content
            except Exception as exc:
                print(f"Request at position {position} generated an exception: {exc}")
                responses[position] = None  # Or handle the exception as appropriate
        
    return responses

# Run Evaluations

Lets get validation results. For this notebook, we're just looking to calculate correctness. Expect the calls to take ~30 seconds since the model is output it's reasoning as well as whether it's correct or not

In [ ]:
# Helper conversation
def conversation_to_str(conversation: list[dict]) -> str:
    return ''.join([f"{c['type']}: {c['text']}" for c in conversation])

# Construct grader prompts from the chat conversations
grader_prompts = []
for i, c in enumerate(chat_conversations):    
    conversation_str: str = conversation_to_str(c)
    prompt: BaseMessage = build_grader_prompt(conversation_str)
    grader_prompts.append(prompt)


# Call Bedrock threaded to speed up getting all our responses. The results should come back in order.
evaluation_results: list[str] = call_threaded(grader_prompts, call_bedrock)


In [ ]:
import re
import json

# Strip out the correctness grade
def extract_correctness(response):
    # Regular expression to extract everything inside of the sumologquery tags
    regex = r'<correctness>(.*?)</correctness>'
    # Perform the regex search
    matches = re.search(regex, response, re.DOTALL)
    # Extract the matched content, if any
    return matches.group(1).strip() if matches else None

# Strip out the reasoning
def extract_reasoning(response):
    # Regular expression to extract everything inside of the sumologquery tags
    regex = r'<thinking>(.*?)</thinking>'
    # Perform the regex search
    matches = re.search(regex, response, re.DOTALL)
    # Extract the matched content, if any
    return matches.group(1).strip() if matches else None
    

def format_results(grade: str, chat_conversation: list[dict]) -> dict:
    reasoning: str = extract_reasoning(grade)
    correctness: str =  extract_correctness(grade)
    
    return {
        'chat_conversation': chat_conversation,
        'reasoning': reasoning,
        'correctness': correctness
    }


formatted_results = [format_results(g, chat_conversations[i]) for i, g in enumerate(evaluation_results)]


In [ ]:
import pandas as pd
evaluated_df = pd.DataFrame(formatted_results)    

# Results

Now that we have our new evaluation dataframe, lets do an analysis on the results

In [ ]:
 # Next lets see how many we got correct
percentage_correct = evaluated_df['correctness'].value_counts(normalize=True)['correct'] * 100
print(f"Percentage correct: {percentage_correct:.2f}%")

# Human Eval
Based on the correctness score, you should have somewhere around ~60%. In the sample dataset, we explicitly made some conversations which would not pass the rubric to show you how human evaluation comes into play. In the section below, we'll subsample ~10 incorrect responses to help understand where the agent is failing.

In [ ]:
# Lastly we need to do some human evaluation. Lets sample a subsection of 10 incorrect responses
from IPython.display import display, HTML

# Assuming you have a dataframe called 'df' with a column called 'result'
incorrect_rows = evaluated_df[evaluated_df['correctness'] == 'incorrect'].sample(n=10)

from IPython.display import display, HTML

# Convert the dataframe to an HTML table
table_html = incorrect_rows.to_html(index=False, classes='table table-striped')

# Display the HTML table
display(HTML(table_html))

# Next Steps

Now that you've run through this notebook. 
* Go back and play with the rubric. 
* You can also play with the temperature and other hyperparameters of the model to see how that has an effect on your score.

# Conclusion
In this notebook we did a basic qualitative evaluation with an LLM. The dataset provided intentionally has some conversations that would not pass the evaulation to show how a combination of "model as a judge" and human evaluation are needed to understand performance, diagnose issues, and understand what needs to change in the system to improve the overall user experience